In [2]:
import pandas as pd
import plotly.io as pio
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import ipywidgets as w
from IPython.display import display

input_df = pd.read_csv('data/Tracker Quotes 2018-2021 v2.csv', 
                       encoding = "utf8", thousands = ",")
#convert string to date-time format to enable date related analysis
input_df["Date"] = pd.to_datetime(input_df["Date"]).dt.strftime("%m-%d-%Y")
convert_dict = {"System Size (kW)": float}
input_df = input_df.astype(convert_dict)

trace_list = []
for vendor in input_df["Manufacturer"].unique():
    trace_list.append(
        {"x": input_df[input_df["Manufacturer"] == vendor]["System Size (kW)"],
         "y": input_df[input_df["Manufacturer"] == vendor]["Adjusted Material $/W"],
         "type": "scatter",
         "mode": "markers",
         "name": "{}".format(vendor),
         "customdata": np.stack((input_df[input_df["Manufacturer"] == vendor]["Date"],
                                 input_df[input_df["Manufacturer"] == vendor]["Project"]), axis = -1),
         "hovertemplate": '<b> %{customdata[1]} </b>' +\
                          '<br>%{customdata[0]}<br>' +\
                          '<br>%{x:.2f} kW' +\
                          '<br>$%{y:.3f} / W'
        }
    )

layout = dict(title = "Material $/W vs System Size (kW-DC)",
              title_x = 0.5,
              xaxis = dict(title = "System kW-DC",
                           tickformat = "digits"),
              yaxis = dict(title = "Racking Material $/W")
             )

fig = go.FigureWidget(data = trace_list, layout = layout)

def update_fig(change):
    aux_df = input_df[input_df["Coast"].isin(change["new"])]
    with fig.batch_update():
        for trace, vendor in zip(fig.data, [x for x in input_df["Manufacturer"].unique()]):
            trace.x = aux_df[aux_df["Manufacturer"] == vendor]["System Size (kW)"]
            trace.y = aux_df[aux_df["Manufacturer"] == vendor]["Adjusted Material $/W"]

drop = w.Dropdown(options =[
    ("All", ["West", "East"]),
    ("West", ["West"]),
    ("East", ["East"]),
])

drop.observe(update_fig, names = "value")
display(w.VBox([drop, fig]))